In [1]:
import pandas as pd
import numpy as np
import os
import ddf_utils.ddf_reader as ddf

In [2]:
ddf.SEARCH_PATH = "/Users/semio/src/work/Gapminder/"

In [3]:
l = pd.read_csv('./GW_indicators_list.csv')

In [4]:
l2 = l[['gw_ddf_id', 'found_in_ddf', 'ddf_concept_id', 'Notes']]
l2 = l2[~l2['found_in_ddf'].isnull()]

In [16]:
geo = ddf.ddf_entities('ddf--gapminder--geo_entity_domain')['country']

In [15]:
def get_geo_map(geo_2, column='area'):

    geo_map = {} 

    cols = ['alternative_1', 'alternative_2', 'alternative_3', 'gapminder_list',
           'alternative_4_cdiac', 'pandg', 'god_id', 'alt_5', 'upper_case_name',
           'code', 'arb1', 'arb2', 'arb3', 'arb4', 'arb5', 'arb6', 'name']

    for i, v in geo_2.iterrows():

        geo_map[v[column]] = np.nan


        for c in cols:
            f = geo[geo[c] == v['name'].strip()]
            if len(f) > 0:
                geo_map[v[column]] = f['country'].values[0]
            if len(f) > 1:
                print(f)
                
    return geo_map

In [5]:
d = iter(l2.groupby('found_in_ddf').groups.items())

In [6]:
len(l2.groupby('found_in_ddf').groups.items())

37

In [7]:
it = next(d)

In [8]:
l2.ix[it[1]]

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
44,data_quality_life_expectancy,ddf--g-hist_lex,data_quality,NaN


In [17]:
geo2 = ddf.ddf_entities(it[0])['area']

In [19]:
geo_map = get_geo_map(geo2)

In [21]:
a = pd.DataFrame(data=geo_map, index=['map']).T
a = a[a['map'].isnull()]

geo2.set_index('area').ix[a.index]

,name


In [10]:
name = l2.ix[it[1]].iloc[0].gw_ddf_id
name_2 = l2.ix[it[1]].iloc[0].ddf_concept_id

In [11]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [22]:
right = ddf.ddf_datapoint(it[0], name_2)
right.area = right.area.map(lambda x: geo_map[x])
right.columns = ['geo', 'time', name]

In [25]:
right[['geo', name, 'time']]\
.to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)

In [26]:
it = next(d)

In [27]:
l2.ix[it[1]]

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
125,life_expectancy_at_birth_data_from_ihme,ddf--lhme_lex,mean,duplicated
126,life_expectancy_at_birth_with_projections,ddf--lhme_lex,mean,duplicated
127,life_expectancy_years,ddf--lhme_lex,mean,NaN


In [128]:
name = l2.ix[it[1]].iloc[0].gw_ddf_id
name_2 = l2.ix[it[1]].iloc[0].ddf_concept_id

In [108]:
name

'life_expectancy_years'

In [43]:
geo2 = ddf.ddf_entities('ddf--ihme-lex')['location_id']

In [47]:
geo2.columns = ['location_id', 'location_code', 'name']

In [48]:
geo_map = get_geo_map(geo2, 'location_id')

In [50]:
a = pd.DataFrame(data=geo_map, index=['map']).T
a = a[a['map'].isnull()]

geo2.set_index('location_id').ix[a.index]

,location_code,name
1,G,Global
2,D0,Developing
3,D1,Developed
4,S6,"Southeast Asia, East Asia, and Oceania"
5,R3,East Asia
9,R5,Southeast Asia
21,R17,Oceania
31,S2,"Central Europe, Eastern Europe, and Central Asia"
32,R2,Central Asia
42,R8,Central Europe


In [54]:
geo_map[171] = 'cod'

a = a.drop(171)

In [109]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [129]:
right = ddf.ddf_datapoint('ddf--ihme-lex', name_2)

right = right.query("location_id not in @a.index & sex_id == 3 & age_group_id == 5")

right.location_id = right.location_id.map(lambda x: geo_map[x])

In [130]:
right = right[['location_id', 'year', 'mean']]
right.columns = ['geo', 'time', name]

In [131]:
# r = right.groupby(['location_id', 'year']).agg(np.mean)[['mean']]
right = right.set_index(['geo', 'time']).sort_index()

In [132]:
right.reset_index()[[name, 'geo', 'time']]\
.to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)

In [133]:
it = next(d)
l2.ix[it[1]]

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
48,old_version_of_income_per_person_version_8,ddf--gapminder--hist_gdppc,gdp_per_capita_by_purchasing_power_parities,NaN
49,subsistence_incomes_per_person,ddf--gapminder--hist_gdppc,gdp_per_capita_by_purchasing_power_parities,divide by 456.25
50,old_version_of_income_per_person_version_3,ddf--gapminder--hist_gdppc,gdp_per_capita_by_purchasing_power_parities,data difference?
51,income_per_person_gdppercapita_ppp_inflation_a...,ddf--gapminder--hist_gdppc,gdp_per_capita_by_purchasing_power_parities,data difference?
52,income_per_person_long_series,ddf--gapminder--hist_gdppc,gdp_per_capita_by_purchasing_power_parities,data difference?
53,income_per_person_with_projections,ddf--gapminder--hist_gdppc,gdp_per_capita_by_purchasing_power_parities,NaN


In [179]:
name = l2.ix[it[1]].iloc[5].gw_ddf_id
name_2 = l2.ix[it[1]].iloc[5].ddf_concept_id

In [156]:
geo2 = ddf.ddf_entities(it[0])['area']
geo_map = get_geo_map(geo2, 'area')

In [157]:
a = pd.DataFrame(data=geo_map, index=['map']).T
a = a[a['map'].isnull()]

geo2.set_index('area').ix[a.index]

,name
nagorno_karabakh,Nagorno-Karabakh


In [158]:
geo_map['nagorno_karabakh'] = 'ngokar'

In [180]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [181]:
right = ddf.ddf_datapoint(it[0], name_2)
right.columns = ['geo', 'time', name]
right.geo = right.geo.map(lambda x: geo_map[x])

In [153]:
# right = right.set_index(['geo', 'time'])
# right = right / 456.25
# right = right.reset_index()

In [185]:
right[['geo', name, 'time']].to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)

In [186]:
it = next(d)
l2.ix[it[1]]

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
190,crude_birth_rate_births_per_1000_population,ddf--unpop-wpp_demog,crude_birth_rate,NaN
191,crude_death_rate_deaths_per_1000_population,ddf--unpop-wpp_demog,crude_death_rate,NaN
192,life_expectancy_female,ddf--unpop-wpp_demog,life_expectancy_at_birth_females,NaN
193,life_expectancy_male,ddf--unpop-wpp_demog,life_expectancy_at_birth_males,NaN
194,population_growth_annual_percent_with_projections,ddf--unpop-wpp_demog,population_growth_rate,NaN


In [228]:
name = l2.ix[it[1]].iloc[4].gw_ddf_id
name_2 = l2.ix[it[1]].iloc[4].ddf_concept_id

In [229]:
name

'population_growth_annual_percent_with_projections'

In [210]:
geo2 = ddf.ddf_entities(it[0])['country_code']

geo2.columns = ['geo', 'name']

geo_map = get_geo_map(geo2, 'geo')

In [211]:
a = pd.DataFrame(data=geo_map, index=['map']).T
a = a[a['map'].isnull()]

geo2.set_index('geo').ix[a.index]

,name
158,Other non-specified areas
180,Democratic Republic of the Congo
364,Iran (Islamic Republic of)
410,Republic of Korea
418,Lao People's Democratic Republic
834,United Republic of Tanzania
900,WORLD
901,More developed regions
902,Less developed regions
903,AFRICA


In [212]:
geo_map[180] = 'cod'
geo_map[364] = 'irn'
geo_map[410] = 'kor'
geo_map[834] = 'tza'
geo_map[957] = 'pyf'

a = a.drop([180, 364, 410, 834, 957])

In [230]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [231]:
right = ddf.ddf_datapoint(it[0], name_2)

right = right.query("country_code not in @a.index")[['country_code', 'reference_date_1_january_31_december', name_2]]
right.columns = ['geo', 'time', name]

right.geo = right.geo.map(lambda x: geo_map[x])

In [233]:
right.sort_values(by=['geo', 'time'])[['geo', name, 'time']]\
.to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)

In [234]:
it = next(d)
l2.ix[it[1]]

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
89,gdpperworking_hour_us_inflation_adjusted,ddf--ilo--kilm_labour_prod,gdp_per_hour_worked_constant_1990_us_at_ppp,data different?


In [236]:
name = l2.ix[it[1]].iloc[0].gw_ddf_id
name_2 = l2.ix[it[1]].iloc[0].ddf_concept_id

In [237]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [238]:
right = ddf.ddf_datapoint(it[0], name_2)

In [241]:
right.columns = ['geo', 'time', name]

In [242]:
right.sort_values(by=['geo', 'time'])[['geo', name, 'time']]\
.to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)

In [243]:
it = next(d)
l2.ix[it[1]]

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
144,corruption_perception_index_cpi,ddf--transpint--corrupton,cpi,combine with old data


In [244]:
name = l2.ix[it[1]].iloc[0].gw_ddf_id
name_2 = l2.ix[it[1]].iloc[0].ddf_concept_id

In [245]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [246]:
right = ddf.ddf_datapoint(it[0], name_2)

In [250]:
right.columns = ['geo', 'time', name]
right = right.set_index(['geo', 'time'])

In [257]:
final = left.append(right)

In [259]:
final.sort_index().reset_index()[['geo', name, 'time']]\
.to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)

In [260]:
it = next(d)
l2.ix[it[1]]

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
145,literacy_rate_adult_female_percent_of_females_...,ddf--uis--stats,lr_ag15t99_f,NaN
146,literacy_rate_adult_male_percent_of_males_ages...,ddf--uis--stats,lr_ag15t99_m,NaN
147,literacy_rate_youth_female_percent_of_females_...,ddf--uis--stats,lr_ag15t24_f,NaN
148,literacy_rate_youth_male_percent_of_males_ages...,ddf--uis--stats,lr_ag15t24_m,NaN
149,literacy_rate_youth_total_percent_of_people_ag...,ddf--uis--stats,lr_ag15t24,NaN
150,literacy_rate_adult_total_percent_of_people_ag...,ddf--uis--stats,lr_ag15t99,NaN


In [316]:
name = l2.ix[it[1]].iloc[5].gw_ddf_id
name_2 = l2.ix[it[1]].iloc[5].ddf_concept_id

In [291]:
geo2 = ddf.ddf_entities(it[0])['location']

In [268]:
geo_map = get_geo_map(geo2, 'location')

In [274]:
a = pd.DataFrame(data=geo_map, index=['map']).T
a = a[a['map'].isnull()]

geo2.set_index('location').ix[a.index].ix[40:]

,name
40520,Latin America and the Caribbean
40525,Arab States
40530,Central and Eastern Europe
40535,South and West Asia
40540,Sub-Saharan Africa
ala,Åland Islands
blm,Saint-Barthélemy
cod,Democratic Republic of the Congo
efa,Education For All Regions
hkg,"China, Hong Kong Special Administrative Region"


In [275]:
to_drop = []

for i in a.index:
    if i in geo.country.values:
        to_drop.append(i)

In [277]:
a = a.drop(to_drop)

In [310]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [311]:
right = ddf.ddf_datapoint(it[0], name_2)

In [312]:
right = right.query("location not in @a.index")

In [313]:
right.columns = ['geo', 'time', name]

In [314]:
right[['geo', name, 'time']].to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)

In [317]:
it = next(d)
l2.ix[it[1]]

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
70,agriculture_workers_percent_of_labour_force,ddf--ilo--kilm_employment_sector,agriculture_percentage,filter sex=='mf'
71,female_agriculture_workers_percent_of_female_l...,ddf--ilo--kilm_employment_sector,agriculture_percentage,filter sex=='f'
72,female_industry_workers_percent_of_female_labo...,ddf--ilo--kilm_employment_sector,industry_percentage,NaN
73,female_service_workers_percent_of_female_labou...,ddf--ilo--kilm_employment_sector,services_percentage,NaN
74,industry_workers_percent_of_labour_force,ddf--ilo--kilm_employment_sector,industry_percentage,NaN
75,male_agriculture_workers_percent_of_male_labou...,ddf--ilo--kilm_employment_sector,agriculture_percentage,NaN
76,male_industry_workers_percent_of_male_labour_f...,ddf--ilo--kilm_employment_sector,industry_percentage,NaN
77,male_service_workers_percent_of_male_labour_force,ddf--ilo--kilm_employment_sector,services_percentage,NaN
78,service_workers_percent_of_labour_force,ddf--ilo--kilm_employment_sector,services_percentage,NaN


In [383]:
name = l2.ix[it[1]].iloc[8].gw_ddf_id
name_2 = l2.ix[it[1]].iloc[8].ddf_concept_id

In [378]:
name

'service_workers_percent_of_labour_force'

In [358]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [379]:
right = ddf.ddf_datapoint(it[0], name_2)
right = right.query("sex == 'mf'")[['country', 'year', name_2]]

In [380]:
right.columns = ['geo', 'time', name]

In [381]:
right[['geo', name, 'time']].to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)

In [384]:
it = next(d)
l2.ix[it[1]]

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
26,agricultural_water_withdrawal_percent_of_total,ddf--fao--aquastat,agricultural_water_withdrawal_as_of_total_wate...,NaN
27,desalinated_water_produced_billion_cu_meters,ddf--fao--aquastat,desalinated_water_produced,NaN
28,industrial_water_withdrawal_percent_of_total,ddf--fao--aquastat,industrial_water_withdrawal_as_of_total_water_...,NaN
29,municipal_water_withdrawal_percent_of_total,ddf--fao--aquastat,municipal_water_withdrawal_as_of_total_withdrawal,NaN
30,internal_renewable_water_cu_meters_per_person,ddf--fao--aquastat,total_internal_renewable_water_resources_per_c...,data difference?
31,renewable_water_cu_meters_per_person,ddf--fao--aquastat,total_renewable_water_resources_per_capita,data difference?
32,total_water_withdrawal_billion_cu_meters,ddf--fao--aquastat,total_water_withdrawal,NaN
33,water_withdrawal_cu_meters_per_person,ddf--fao--aquastat,total_water_withdrawal_per_capita,NaN


In [433]:
name = l2.ix[it[1]].iloc[7].gw_ddf_id
name_2 = l2.ix[it[1]].iloc[7].ddf_concept_id

In [427]:
name

'water_withdrawal_cu_meters_per_person'

In [428]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [429]:
right = ddf.ddf_datapoint(it[0], name_2)

In [430]:
right.columns = ['geo', 'time', name]

In [431]:
right[['geo', name, 'time']].to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)

In [434]:
it = next(d)
l2.ix[it[1]]

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
195,math_achievement_4th_grade,ddf--wb--edu,lo_timss_mat4,NaN
196,math_achievement_8th_grade,ddf--wb--edu,lo_timss_mat8,NaN


In [445]:
name = l2.ix[it[1]].iloc[1].gw_ddf_id
name_2 = l2.ix[it[1]].iloc[1].ddf_concept_id

In [437]:
name

'math_achievement_4th_grade'

In [446]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [447]:
right = ddf.ddf_datapoint(it[0], name_2)

In [448]:
right.columns = ['time', 'geo', name]

In [449]:
right = right.sort_values(by=['geo', 'time'])

In [452]:
right[['geo', name, 'time']].to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)

In [453]:
it = next(d)
l2.ix[it[1]]

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
60,population_total,ddf--gapminder--hist_pop,total_population,NaN


In [457]:
name = l2.ix[it[1]].iloc[0].gw_ddf_id
name_2 = l2.ix[it[1]].iloc[0].ddf_concept_id

In [487]:
geo2 = ddf.ddf_entities(it[0])['area']
geo_map = get_geo_map(geo2, 'area')

In [488]:
a = pd.DataFrame(data=geo_map, index=['map']).T
a = a[a['map'].isnull()]

geo2.set_index('area').ix[a.index]

,name


In [458]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [459]:
right = ddf.ddf_datapoint(it[0], name_2)

In [463]:
right.columns = ['geo', 'time', name]

right.geo = right.geo.map(lambda x: geo_map[x])

In [465]:
right[['geo', name, 'time']].to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.0f', index=False)

In [467]:
it = next(d)
l2.ix[it[1]]

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
79,female_family_workers_percent_of_female_labour...,ddf--ilo--kilm_employment_status,contributing_family_workers_percentage,NaN
80,family_workers_percent_of_labour_force,ddf--ilo--kilm_employment_status,contributing_family_workers_percentage,NaN
81,female_salaried_workers_percent_of_female_labo...,ddf--ilo--kilm_employment_status,wage_salaried_workers_employees_percentage,NaN
82,female_self_employed_percent_of_female_labour_...,ddf--ilo--kilm_employment_status,total_self_employed_workers_percentage,NaN
83,male_family_workers_percent_of_male_labour_force,ddf--ilo--kilm_employment_status,contributing_family_workers_percentage,NaN
84,male_salaried_workers_percent_of_male_labour_f...,ddf--ilo--kilm_employment_status,wage_salaried_workers_employees_percentage,NaN
85,male_self_employed_percent_of_male_labour_force,ddf--ilo--kilm_employment_status,total_self_employed_workers_percentage,NaN
86,salaried_workers_percent_of_labour_force,ddf--ilo--kilm_employment_status,wage_salaried_workers_employees_percentage,NaN
87,self_employed_percent_of_labour_force,ddf--ilo--kilm_employment_status,total_self_employed_workers_percentage,NaN


In [468]:
name = l2.ix[it[1]].iloc[0].gw_ddf_id
name_2 = l2.ix[it[1]].iloc[0].ddf_concept_id

In [469]:
name

'female_family_workers_percent_of_female_labour_force'

In [470]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [474]:
right = ddf.ddf_datapoint(it[0], name_2)
right = right.query("sex == 'f'")[['country', 'year', name_2]]

In [475]:
right.columns = ['geo', 'time', name]

In [477]:
right[['geo', name, 'time']].to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)

In [479]:
for i, v in l2.ix[it[1]].iterrows():
    name = v['gw_ddf_id']
    name_2 = v['ddf_concept_id']
    right = ddf.ddf_datapoint(it[0], name_2)
    if 'female' in name:
        q = 'f'
    elif 'male' in name:
        q = 'm'
    else:
        q = 'mf'
    right = right.query("sex == @q")[['country', 'year', name_2]]
    right.columns = ['geo', 'time', name]
    (right[['geo', name, 'time']]
     .to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False))

In [480]:
it = next(d)
l2.ix[it[1]]

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
151,annual_hiv_deaths_number_all_ages,ddf--unaids--aidsinfo,aids_related_deaths_all_ages_estimate,data different?
152,art_coverage_percent_cd4_l_350,ddf--unaids--aidsinfo,coverage_of_people_receiving_art_total_estimate,data different?
153,newly_hiv_infected_number_all_ages,ddf--unaids--aidsinfo,new_hiv_infections_all_ages_estimate,data different?
154,newly_hiv_infected_percent_age_15_49,ddf--unaids--aidsinfo,new_hiv_infections_adults_15_estimate,NaN


In [508]:
geo2 = ddf.ddf_entities(it[0])['area']
# geo2 = geo2[geo2.area.str.match("^[a-z_]+$")]
geo_map = get_geo_map(geo2, 'area')

In [509]:
a = pd.DataFrame(data=geo_map, index=['map']).T
a = a[a['map'].isnull()]

geo2.set_index('area').ix[a.index]

,name
03m49wld,All countries
arm001,Aragatsotn
arm002,Ararat
arm003,Armavir
arm005,Gegharkunik
arm008,Kotaik
arm009,Lori
arm013,Shirak
arm015,Syunik
arm016,Tavush


In [510]:
to_drop = []

for i in a.index:
    if i in geo.country.values:
        to_drop.append(i)

In [511]:
a = a.drop(to_drop)

In [538]:
name = l2.ix[it[1]].iloc[3].gw_ddf_id
name_2 = l2.ix[it[1]].iloc[3].ddf_concept_id

In [532]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [533]:
right = ddf.ddf_datapoint(it[0], name_2)

In [534]:
right = right.query("area not in @a.index")

In [535]:
right.columns = ['time', 'geo', name]

In [536]:
right[[name, 'geo', 'time']].to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%g', index=False)

In [540]:
it = next(d)
l2.ix[it[1]]

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
65,aged_15_24_employment_rate_percent,ddf--ilo--kilm_employment_pop,employment_to_population_ratio,data different?
66,aged_15plus_employment_rate_percent,ddf--ilo--kilm_employment_pop,employment_to_population_ratio,data different?
67,females_aged_15_24_employment_rate_percent,ddf--ilo--kilm_employment_pop,employment_to_population_ratio,data different?
68,males_aged_15_24_employment_rate_percent,ddf--ilo--kilm_employment_pop,employment_to_population_ratio,data different?
69,males_aged_15plus_employment_rate_percent,ddf--ilo--kilm_employment_pop,employment_to_population_ratio,data different?


In [574]:
name = l2.ix[it[1]].iloc[4].gw_ddf_id
name_2 = l2.ix[it[1]].iloc[4].ddf_concept_id

In [569]:
name

'males_aged_15plus_employment_rate_percent'

In [543]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [570]:
right = ddf.ddf_datapoint(it[0], name_2)
right = right.query("sex == 'm' & age_group == 'adult'")[['country', 'year', name_2]]

In [571]:
right.columns = ['geo', 'time', name]

In [572]:
right[['geo', name, 'time']].to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)

In [576]:
it = next(d)
l2.ix[it[1]]

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
118,hourly_compensation_us,ddf--ilo--kilm_wage_compensation,hourly_compensation_costs,data different?


In [578]:
name = l2.ix[it[1]].iloc[0].gw_ddf_id
name_2 = l2.ix[it[1]].iloc[0].ddf_concept_id

In [579]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [580]:
right = ddf.ddf_datapoint(it[0], name_2)

In [582]:
right.columns = ['geo', 'time', name]

In [583]:
right[['geo', name, 'time']].to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)

In [606]:
it = next(d)
l2.ix[it[1]]

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
88,working_hours_per_week,ddf--ilo--kilm_hours_work,annual_number_of_hours_actually_worked_per_person,divide by 52


In [607]:
name = l2.ix[it[1]].iloc[0].gw_ddf_id
name_2 = l2.ix[it[1]].iloc[0].ddf_concept_id

In [608]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [610]:
right = ddf.ddf_datapoint(it[0], name_2)

In [612]:
right.columns = ['geo', 'time', name]

In [614]:
right[name] = right[name] / 52.0

In [615]:
right[['geo', name, 'time']].to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)

In [616]:
it = next(d)
l2.ix[it[1]]

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
11,newborn_deaths,ddf--cme,neonatal_deaths_median,NaN
12,newborn_mortality_rate_per_1000,ddf--cme,newborn_mortality_rate_per_1000,NaN
13,under_five_mortality_from_cme_per_1000_born,ddf--cme,under_five_mortality_from_cme_per_1000_born,NaN


In [645]:
name = l2.ix[it[1]].iloc[2].gw_ddf_id
# name_2 = 'nmr_median'
name_2 = 'u5mr_median'

In [637]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [646]:
right = ddf.ddf_datapoint(it[0], name_2)

In [647]:
right.columns = ['geo', 'time', name]

In [649]:
right[['geo', name, 'time']].to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)

In [650]:
it = next(d)
l2.ix[it[1]]

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
58,maternal_deaths_total_number,ddf--gapminder--hist_maternal_mort,maternal_deaths,data different?
59,maternal_mortality_ratio_per_100000_live_births,ddf--gapminder--hist_maternal_mort,mmr,NaN


In [657]:
geo2 = ddf.ddf_entities(it[0])['country']
geo_map = get_geo_map(geo2, 'country')

In [660]:
a = pd.DataFrame(data=geo_map, index=['map']).T
a = a[a['map'].isnull()]

geo2.set_index('country').ix[a.index]

,name


In [667]:
name = l2.ix[it[1]].iloc[1].gw_ddf_id
name_2 = l2.ix[it[1]].iloc[1].ddf_concept_id

In [654]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [668]:
right = ddf.ddf_datapoint(it[0], name_2)

In [669]:
right.columns = ['geo', 'time', name]

right.geo = right.geo.map(lambda x: geo_map[x])

In [671]:
right[['geo', name, 'time']].to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)

In [673]:
it = next(d)
l2.ix[it[1]]

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
119,body_mass_index_bmi_men_kgperm2,ddf--imperial--metabolic,bmi_mean_kg_per_m2,filter sex=='male'
120,body_mass_index_bmi_women_kgperm2,ddf--imperial--metabolic,bmi_mean_kg_per_m2,filter sex=='female'
121,blood_pressure_sbp_men_mmhg,ddf--imperial--metabolic,sbp_mean_mm_hg,filter sex=='male'
122,blood_pressure_sbp_women_mmhg,ddf--imperial--metabolic,sbp_mean_mm_hg,filter sex=='female'
123,cholesterol_fat_in_blood_men_mmolperl,ddf--imperial--metabolic,tc_mean_mmol_per_l,filter sex=='male'
124,cholesterol_fat_in_blood_women_mmolperl,ddf--imperial--metabolic,tc_mean_mmol_per_l,filter sex=='female'


In [679]:
geo2 = ddf.ddf_entities(it[0])['country']
geo_map = get_geo_map(geo2, 'country')

In [680]:
a = pd.DataFrame(data=geo_map, index=['map']).T
a = a[a['map'].isnull()]

geo2.set_index('country').ix[a.index]

,name
china_hong_kong_sar,China (Hong Kong SAR)
china_macao_sar,China (Macao SAR)
democratic_republic_of_the_congo,Democratic Republic of the Congo
iran_islamic_republic_of,Iran (Islamic Republic of)
lao_people_s_democratic_republic,Lao People's Democratic Republic
macedonia_former_yugoslav_republic_of,Macedonia (Former Yugoslav Republic of)
republic_of_korea,Republic of Korea
united_republic_of_tanzania,United Republic of Tanzania


In [681]:
geo_map['china_hong_kong_sar'] = 'hkg'
geo_map['china_macao_sar'] = 'mac'
geo_map['democratic_republic_of_the_congo'] = 'cod'
geo_map['iran_islamic_republic_of'] = 'irn'
geo_map['lao_people_s_democratic_republic'] = 'lao'
geo_map['macedonia_former_yugoslav_republic_of'] = 'mkd'
geo_map['republic_of_korea'] = 'kor'
geo_map['united_republic_of_tanzania'] = 'tza'

In [674]:
name = l2.ix[it[1]].iloc[1].gw_ddf_id
name_2 = l2.ix[it[1]].iloc[1].ddf_concept_id

In [678]:
name

'body_mass_index_bmi_women_kgperm2'

In [675]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [676]:
right = ddf.ddf_datapoint(it[0], name_2)

In [682]:
right = right.query("sex == 'female'")[['country', 'year', name_2]]

In [683]:
right.columns = ['geo', 'time', name]
right.geo = right.geo.map(lambda x: geo_map[x])

In [686]:
right[[name, 'geo', 'time']].to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)

In [687]:
for i, v in l2.ix[it[1]].iterrows():
    name = v['gw_ddf_id']
    name_2 = v['ddf_concept_id']
    right = ddf.ddf_datapoint(it[0], name_2)
    
    if 'women' in name:
        q = 'female'
    elif 'men' in name:
        q = 'male'
        
    right = right.query("sex == @q")[['country', 'year', name_2]]
    
    right.columns = ['geo', 'time', name]
    right.geo = right.geo.map(lambda x: geo_map[x])
    
    right[[name, 'geo', 'time']].to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)
    

In [688]:
it = next(d)
l2.ix[it[1]]  #TODO: skip this one for now

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
7,co2_emissions_tonnes_per_person,ddf--cdiac-co2,per_capita_co2_emissions_metric_tons_of_carbon,Multiply by 3.667 to get the CO2 data
8,cumulative_co2_emissions_tonnes,ddf--cdiac-co2,total_co2_emissions_from_fossil_fuels_thousand...,the cumulative sum of yearly_co2_emissions_tonnes
9,yearly_co2_emissions_1000_tonnes,ddf--cdiac-co2,total_co2_emissions_from_fossil_fuels_thousand...,equals yearly_co2_emissions_tonnes / 1000
10,yearly_co2_emissions_tonnes,ddf--cdiac-co2,total_co2_emissions_from_fossil_fuels_thousand...,Multiply by 3.667 to get the CO2 data


In [735]:
geo2 = right[['nation']].drop_duplicates()

In [736]:
from ddf_utils.str import to_concept_id

geo2['name'] = geo2['nation'].str.title()
geo2['nation'] = geo2['nation'].map(to_concept_id)

In [737]:
# geo2 = ddf.ddf_entities(it[0])['nation']
geo_map = get_geo_map(geo2, 'nation')

In [738]:
a = pd.DataFrame(data=geo_map, index=['map']).T
a = a[a['map'].isnull()]

geo3 = geo2.set_index('nation').ix[a.index]

In [740]:
geo3

,name
antarctic_fisheries,Antarctic Fisheries
east_west_pakistan,East & West Pakistan
federal_republic_of_germany,Federal Republic Of Germany
federated_states_of_micronesia,Federated States Of Micronesia
federation_of_malaya_singapore,Federation Of Malaya-Singapore
former_democratic_yemen,Former Democratic Yemen
former_german_democratic_republic,Former German Democratic Republic
former_yemen,Former Yemen
french_equatorial_africa,French Equatorial Africa
french_indo_china,French Indo-China


In [698]:
name = l2.ix[it[1]].iloc[0].gw_ddf_id
name_2 = l2.ix[it[1]].iloc[0].ddf_concept_id

In [699]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [705]:
right = ddf.ddf_datapoint(it[0], name_2)

In [706]:
right.version.unique()

array(['1751_2008', '1751_2009', '1751_2010', '1751_2011'], dtype=object)

In [709]:
right = right[right.version == '1751_2011']

In [741]:
it = next(d)
l2.ix[it[1]]

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
57,age_at_1st_marriage_women,ddf--gapminder--hist_marriage_age,average_age_at_1st_marriage_girls,NaN


In [748]:
geo2 = ddf.ddf_entities(it[0])['country']
geo_map = get_geo_map(geo2, 'country')

In [750]:
a = pd.DataFrame(data=geo_map, index=['map']).T
a = a[a['map'].isnull()]

geo2.set_index('country').ix[a.index]

,name
central_african_rep,Central African rep.
czech_rep,Czech rep.
dominican_rep,Dominican rep.
macao_china,"macao, China"
vietnam,VietNam


In [751]:
geo_map['vietnam'] = 'vnm'
geo_map['macao_china'] = 'mac'
geo_map['czech_rep'] = 'cze'
geo_map['dominican_rep'] = 'dom'
geo_map['central_african_rep'] = 'caf'

In [742]:
name = l2.ix[it[1]].iloc[0].gw_ddf_id
name_2 = l2.ix[it[1]].iloc[0].ddf_concept_id

In [743]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [744]:
right = ddf.ddf_datapoint(it[0], name_2)

In [752]:
right.columns = ['geo', 'time', name]

right.geo = right.geo.map(lambda x: geo_map[x])

In [754]:
right[['geo', name, 'time']].to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)

In [762]:
it = next(d)
l2.ix[it[1]]

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
140,alternative_gdppercapita_ppp_inflation_adjuste...,ddf--pwt--penn_world_table,rgdpna / pop,data difference?
141,alternative_gdp_per_capita_ppp_pwt_7_1,ddf--pwt--penn_world_table,NaN,newest version is 9.0
142,alternative_gdp_per_capita_ppp_pwt_8_0,ddf--pwt--penn_world_table,NaN,newest version is 9.0
143,economic_growth_over_the_past_10_years,ddf--pwt--penn_world_table,rgdpna,data difference?


In [778]:
name = l2.ix[it[1]].iloc[3].gw_ddf_id
name_2 = l2.ix[it[1]].iloc[3].ddf_concept_id

In [764]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [779]:
right1 = ddf.ddf_datapoint(it[0], 'rgdpna').set_index(['country', 'year'])
right2 = ddf.ddf_datapoint(it[0], 'pop').set_index(['country', 'year'])

In [789]:
right = pd.concat([right1, right2], axis=1)

In [792]:
right = right.reset_index()

In [802]:
to_concat = []

for c, ids in right.groupby('country').groups.items():
    df = right.ix[ids]
    new_df = cal(df.set_index(['country', 'year']).sort_index())
    to_concat.append(new_df['rgdpna'].reset_index())

In [804]:
r = pd.concat(to_concat)

In [799]:
def cal(x):
    return (x - x.shift(10)) / x.shift(10) * 100

In [815]:
r = r.dropna(how='any').copy()

In [816]:
r.columns = ['geo', 'time', name]

In [817]:
r[name] = r[name] / 10

In [775]:
right = right[name]

In [776]:
right = right.reset_index()
right.columns = ['geo', 'time', name]

In [777]:
right[['geo', name, 'time']].to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)

In [819]:
r.sort_values(by=['geo', 'time'])[['geo', 'time', name]].to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)

In [820]:
it = next(d)
l2.ix[it[1]]

,gw_ddf_id,found_in_ddf,ddf_concept_id,Notes
56,infant_mortality_rate_per_1000_births,ddf--gapminder--hist_imr,infant_mortality_rate,NaN


In [827]:
geo2 = ddf.ddf_entities(it[0])['area']
geo_map = get_geo_map(geo2, 'area')

In [829]:
a = pd.DataFrame(data=geo_map, index=['map']).T
a = a[a['map'].isnull()]

geo2.set_index('area').ix[a.index]

,name
hawaii,Hawaii
malaya,Malaya


In [822]:
name = l2.ix[it[1]].iloc[0].gw_ddf_id
name_2 = l2.ix[it[1]].iloc[0].ddf_concept_id

In [823]:
left = ddf.ddf_datapoint('ddf--gapminder--systema_globalis', name).set_index(['geo', 'time'])

In [835]:
right = ddf.ddf_datapoint(it[0], name_2)

In [836]:
right = right.query("area not in @a.index")
right.columns = ['geo', 'time', name]

right.geo = right.geo.map(lambda x: geo_map[x])

In [837]:
right[['geo', name, 'time']].to_csv('../ddf--datapoints--'+name+'--by--geo--time.csv', float_format='%.2f', index=False)